In [1]:
import ipykernel
ipykernel.__version__

'6.7.0'

In [2]:
import IPython
IPython.__version__

'8.21.0'

In [3]:
import numpy as np
import math
from align import align_semiglobal, score_semiglobal
import torchaudio
from pathlib import Path

ModuleNotFoundError: No module named 'torchaudio'

In [ ]:
pam = np.load("pam.npy")

In [6]:
t0, tn = 0.9, 1.31
f0, fn = math.floor(t0 / 0.02), math.ceil(tn / 0.02)

query = np.load("./segments/train/395/m1govpedzbnfni-308Q0PEVB8D36MZ3GVRGS47IH4H9I5_530406_195536.npz")
x = query["codes"][query["boundaries"][:-1]]

x0, = np.where(query["boundaries"] <= f0)
x0 = x0[-1]
xn, = np.where(query["boundaries"] >= fn)
xn = xn[0]
x = x[x0-1:xn+1]

In [7]:
x

array([28, 70, 26, 92, 57, 94,  8], dtype=int32)

In [8]:
test = np.load("./segments/train/395/mff8fabdoddts-3ERMJ6L4DYSLOM11MDRO61NT07V7MQ_394974_736211.npz")
y = test["codes"][test["boundaries"][:-1]]

In [9]:
path, p, q, score = align_semiglobal(x, y, pam, 3)

In [10]:
indexes, = np.where(np.array(p) != -1)
start, end = indexes[1], indexes[-1]
norm_score = score / (end - start)
_, b0 = path[start - 1]
_, bT = path[end]
w0, wT = 0.02 * test["boundaries"][b0 - 1], 0.02 * test["boundaries"][bT]
offset = int(w0 * 16000)
frames = int((wT - w0) * 16000)

In [11]:
wav, sr = torchaudio.load("/home/leannenortje/Datasets/spokencoco/SpokenCOCO/wavs/train/395/mff8fabdoddts-3ERMJ6L4DYSLOM11MDRO61NT07V7MQ_394974_736211.wav", frame_offset=offset, num_frames=frames)

In [12]:
from IPython.display import Audio

In [15]:
print(wav.squeeze().numpy().shape)
Audio(wav.squeeze().numpy(), rate=sr)

(9600,)


In [12]:
norm_score

1.4

In [ ]:
x, y

In [ ]:
from tqdm.notebook import tqdm
seg_dir = Path('segments/val')
wav_dir = Path('/home/leannenortje/Datasets/spokencoco/SpokenCOCO/wavs/val')
save_dir = Path('zebra_examples')
for fn in tqdm(list(seg_dir.rglob('*.npz'))):
    test = np.load(fn)
    y = test["codes"][test["boundaries"][:-1]]
    path, p, q, score = align_semiglobal(x, y, pam, 3)
    indexes, = np.where(np.array(p) != -1)
    start, end = indexes[1], indexes[-1]
    norm_score = score / (end - start)
    
    if norm_score >= 1.2:
        
        _, b0 = path[start - 1]
        _, bT = path[end]
        w0, wT = 0.02 * test["boundaries"][b0 - 1], 0.02 * test["boundaries"][bT]
        offset = int(w0 * 16000)
        frames = int((wT - w0) * 16000)
        wav_path = wav_dir / fn.relative_to(seg_dir)
        wav, sr = torchaudio.load(wav_path.with_suffix('.wav'), frame_offset=offset, num_frames=frames)
        out_path = (save_dir / fn.relative_to(seg_dir)).with_suffix('.wav')
        out_path.parent.mkdir(parents=True, exist_ok=True)
        torchaudio.save(out_path, wav, sr)

In [ ]:
align_semiglobal(x, y, pam, 3, start_gap="x", end_gap="x")

In [ ]:
len(x), len(y)

In [ ]:
len(segments["codes"])

In [ ]:
import csv

In [ ]:
with open("/home/benjamin/Documents/align/PAMAUDIO.csv") as file:
    reader = csv.reader(file)
    D = {(int(a), int(b)): float(score) for a, b, score in reader}

    silences = {2, 3, 4, 6, 16, 19, 21, 22, 32, 37, 44, 45, 69, 74, 87, 96}
    for (i, j), v in D.items():
        if i in silences or j in silences:
            value = 0.0 if i in silences and j in silences else -10.0
            D[(i, j)] = value

In [ ]:
pam = np.zeros((100, 100), dtype=int)
for (i, j), v in D.items():
    pam[i, j] = v

In [ ]:
np.save("pam.npy", pam)